In [2]:
import pandas as pd
import numpy as np
import geopandas as gpp
import seaborn as sns

df_votacao_candidato_munzona = pd.read_csv('/Users/willianribeiro/Documents/GitHub/analise_exploratoria_dados_tse/data/Resultados/2020/votacao_candidato_munzona_2020/votacao_candidato_munzona_2020_BRASIL.csv', sep=';', encoding='Latin 1')
df_votacao_partido_munzona = pd.read_csv('/Users/willianribeiro/Documents/GitHub/analise_exploratoria_dados_tse/data/Resultados/2020/votacao_partido_munzona_2020/votacao_partido_munzona_2020_BRASIL.csv', sep=';', encoding='Latin 1')
df_detalhe_votacao_munzona = pd.read_csv('/Users/willianribeiro/Documents/GitHub/analise_exploratoria_dados_tse/data/Resultados/2020/detalhe_votacao_munzona_2020/detalhe_votacao_munzona_2020_BRASIL.csv', sep=';', encoding='Latin 1')

In [3]:
df_votacao_candidato_munzona.columns

Index(['DT_GERACAO', 'HH_GERACAO', 'ANO_ELEICAO', 'CD_TIPO_ELEICAO',
       'NM_TIPO_ELEICAO', 'NR_TURNO', 'CD_ELEICAO', 'DS_ELEICAO', 'DT_ELEICAO',
       'TP_ABRANGENCIA', 'SG_UF', 'SG_UE', 'NM_UE', 'CD_MUNICIPIO',
       'NM_MUNICIPIO', 'NR_ZONA', 'CD_CARGO', 'DS_CARGO', 'SQ_CANDIDATO',
       'NR_CANDIDATO', 'NM_CANDIDATO', 'NM_URNA_CANDIDATO',
       'NM_SOCIAL_CANDIDATO', 'CD_SITUACAO_CANDIDATURA',
       'DS_SITUACAO_CANDIDATURA', 'CD_DETALHE_SITUACAO_CAND',
       'DS_DETALHE_SITUACAO_CAND', 'TP_AGREMIACAO', 'NR_PARTIDO', 'SG_PARTIDO',
       'NM_PARTIDO', 'NR_FEDERACAO', 'NM_FEDERACAO', 'SG_FEDERACAO',
       'DS_COMPOSICAO_FEDERACAO', 'SQ_COLIGACAO', 'NM_COLIGACAO',
       'DS_COMPOSICAO_COLIGACAO', 'ST_VOTO_EM_TRANSITO', 'QT_VOTOS_NOMINAIS',
       'NM_TIPO_DESTINACAO_VOTOS', 'QT_VOTOS_NOMINAIS_VALIDOS',
       'CD_SIT_TOT_TURNO', 'DS_SIT_TOT_TURNO'],
      dtype='object')

In [4]:
df_votacao_partido_munzona.columns

Index(['DT_GERACAO', 'HH_GERACAO', 'ANO_ELEICAO', 'CD_TIPO_ELEICAO',
       'NM_TIPO_ELEICAO', 'NR_TURNO', 'CD_ELEICAO', 'DS_ELEICAO', 'DT_ELEICAO',
       'TP_ABRANGENCIA', 'SG_UF', 'SG_UE', 'NM_UE', 'CD_MUNICIPIO',
       'NM_MUNICIPIO', 'NR_ZONA', 'CD_CARGO', 'DS_CARGO', 'TP_AGREMIACAO',
       'NR_PARTIDO', 'SG_PARTIDO', 'NM_PARTIDO', 'NR_FEDERACAO',
       'NM_FEDERACAO', 'SG_FEDERACAO', 'DS_COMPOSICAO_FEDERACAO',
       'SQ_COLIGACAO', 'NM_COLIGACAO', 'DS_COMPOSICAO_COLIGACAO',
       'ST_VOTO_EM_TRANSITO', 'QT_VOTOS_LEGENDA_VALIDOS',
       'QT_VOTOS_NOMINAIS_CONVR_LEG', 'QT_TOTAL_VOTOS_LEG_VALIDOS',
       'QT_VOTOS_NOMINAIS_VALIDOS', 'QT_VOTOS_LEGENDA_ANUL_SUBJUD',
       'QT_VOTOS_NOMINAIS_ANUL_SUBJUD'],
      dtype='object')

In [5]:
df_detalhe_votacao_munzona.columns

Index(['DT_GERACAO', 'HH_GERACAO', 'ANO_ELEICAO', 'CD_TIPO_ELEICAO',
       'NM_TIPO_ELEICAO', 'NR_TURNO', 'CD_ELEICAO', 'DS_ELEICAO', 'DT_ELEICAO',
       'TP_ABRANGENCIA', 'SG_UF', 'SG_UE', 'NM_UE', 'CD_MUNICIPIO',
       'NM_MUNICIPIO', 'NR_ZONA', 'CD_CARGO', 'DS_CARGO', 'QT_APTOS',
       'QT_SECOES_PRINCIPAIS', 'QT_SECOES_AGREGADAS',
       'QT_SECOES_NAO_INSTALADAS', 'QT_TOTAL_SECOES', 'QT_COMPARECIMENTO',
       'QT_ELEITORES_SECOES_NAO_INSTALADAS', 'QT_ABSTENCOES',
       'ST_VOTO_EM_TRANSITO', 'QT_VOTOS', 'QT_VOTOS_CONCORRENTES',
       'QT_VOTOS_VALIDOS', 'QT_VOTOS_NOMINAIS_VALIDOS',
       'QT_TOTAL_VOTOS_LEG_VALIDOS', 'QT_VOTOS_LEGENDA_VALIDOS',
       'QT_VOTOS_NOMINAIS_CONVR_LEG', 'QT_TOTAL_VOTOS_ANULADOS',
       'QT_VOTOS_NOMINAIS_ANULADOS', 'QT_VOTOS_LEGENDA_ANULADOS',
       'QT_TOTAL_VOTOS_ANUL_SUBJUD', 'QT_VOTOS_NOMINAIS_ANUL_SUBJUD',
       'QT_VOTOS_LEGENDA_ANUL_SUBJUD', 'QT_VOTOS_BRANCOS',
       'QT_TOTAL_VOTOS_NULOS', 'QT_VOTOS_NULOS', 'QT_VOTOS_NULO_TECNICO',


# Sobre as métricas que queremos fornecer:
- Quociente Eleitoral
- Quartil Partidário
- Total de Eleitores por Municipio
- Total de abstenções por Municipio
- Total de votos em branco
- Gráfico com total de votos por candidato
- Gráfico total de votos por partido
- Total de Candidatos Eleitos
- Clausula de Barreira da Eleição

### Quociente Eleitoral 

Definido como Total de Votos validos dividido Pela quantidade de cadeiras disponíveis

In [15]:
def contador_eleitos(dataset_tratado_cidade_cargo):
    dicionario_eleicao =  {
        'SUPLENTE': 0,
        'NÃO ELEITO': 0,
        'ELEITO POR QP': 0,
        'ELEITO POR MÉDIA': 0
    }

    # Conjunto para armazenar nomes de candidatos já encontrados
    nomes_encontrados = set()

    # Iterar pelas linhas do DataFrame
    for index, row in dataset_tratado_cidade_cargo.iterrows():
        # Verificar se o nome do candidato já apareceu
        if row['NM_CANDIDATO'] in nomes_encontrados:
        # Ignora a linha se o nome do candidato ja apareceu
            pass
        else:
            nomes_encontrados.add(row['NM_CANDIDATO'])
            dicionario_eleicao[row['DS_SIT_TOT_TURNO']] += 1

    return dicionario_eleicao

In [17]:
cidade = 'SÃO PAULO'

cargo = 'Vereador'

df_cidade_filtrado = df_votacao_candidato_munzona[df_votacao_candidato_munzona['NM_MUNICIPIO'] == cidade]

df_cidade_filtrado_cargo = df_cidade_filtrado[df_cidade_filtrado['DS_CARGO'] == cargo]

resultados_eleitos = contador_eleitos(df_cidade_filtrado_cargo)

cadeiras = resultados_eleitos['ELEITO POR QP'] + resultados_eleitos['ELEITO POR MÉDIA']

total_votos = df_cidade_filtrado_cargo['QT_VOTOS_NOMINAIS_VALIDOS'].sum()

quociente = total_votos / cadeiras

print(f'Total de cadeiras: {cadeiras}')
print(f'Total de votos: {total_votos}')
print(f'Quociente eleitoral: {quociente}')

Total de cadeiras: 55
Total de votos: 4427182
Quociente eleitoral: 80494.21818181819


### Quociente Partidário

Definido como Total de Votos validos dividido pela quantidade de partidos

In [ ]:
def calculadora_partido(dataset_tratado_cidade_cargo):
    dicionario_partido = {}

    # Conjunto para armazenar nomes de candidatos já encontrados
    partidos_encontrados = set()

    # Iterar pelas linhas do DataFrame
    for index, row in dataset_tratado_cidade_cargo.iterrows():
        # Verificar se o nome do candidato já apareceu
        if row['NM_PARTIDO'] in partidos_encontrados:
            dicionario_partido[row['NM_PARTIDO']] += row['QT_VOTOS_NOMINAIS_VALIDOS']
        else:
            partidos_encontrados.add(row['NM_PARTIDO'])
            dicionario_partido[row['NM_PARTIDO']] = row['QT_VOTOS_NOMINAIS_VALIDOS']

    return dicionario_partido

In [20]:
df_votacao_partido_munzona.columns

Index(['DT_GERACAO', 'HH_GERACAO', 'ANO_ELEICAO', 'CD_TIPO_ELEICAO',
       'NM_TIPO_ELEICAO', 'NR_TURNO', 'CD_ELEICAO', 'DS_ELEICAO', 'DT_ELEICAO',
       'TP_ABRANGENCIA', 'SG_UF', 'SG_UE', 'NM_UE', 'CD_MUNICIPIO',
       'NM_MUNICIPIO', 'NR_ZONA', 'CD_CARGO', 'DS_CARGO', 'TP_AGREMIACAO',
       'NR_PARTIDO', 'SG_PARTIDO', 'NM_PARTIDO', 'NR_FEDERACAO',
       'NM_FEDERACAO', 'SG_FEDERACAO', 'DS_COMPOSICAO_FEDERACAO',
       'SQ_COLIGACAO', 'NM_COLIGACAO', 'DS_COMPOSICAO_COLIGACAO',
       'ST_VOTO_EM_TRANSITO', 'QT_VOTOS_LEGENDA_VALIDOS',
       'QT_VOTOS_NOMINAIS_CONVR_LEG', 'QT_TOTAL_VOTOS_LEG_VALIDOS',
       'QT_VOTOS_NOMINAIS_VALIDOS', 'QT_VOTOS_LEGENDA_ANUL_SUBJUD',
       'QT_VOTOS_NOMINAIS_ANUL_SUBJUD'],
      dtype='object')